# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()

In [23]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [29]:
try:
    driver.find_element_by_id("checkbox").click()
    driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]').click()
except:
    pass

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [25]:
driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a').click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [26]:
dropdown = Select(driver.find_element_by_id("slcJurisdiction"))
dropdown.select_by_visible_text('Statewide')
driver.find_element_by_id('txtTradeName').send_keys("VAP%")

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [27]:
driver.find_element_by_name("searchForm").submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [28]:
searchlistitems = driver.find_elements_by_class_name("searchlistitem")
for item in searchlistitems:
    print(item.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [9]:
items = driver.find_elements_by_class_name("searchfieldtitle")
for item in items:
    print("----")
    title = item.find_element_by_class_name("searchlistitem").text
    print(title)
    try:
        link = item.find_element_by_tag_name("a")
        print(link.get_attribute('href'))
    except:
        print("No link")

----
VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D
----
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=NXzECYG7G0U%3D
----
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=dm%2FAmpglke0%3D
----
VAPE FROG
No link
----
VAPE FROG
No link


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [10]:
items = driver.find_elements_by_class_name("searchfieldtitle")
for item in items:
    print("----")
    title = item.find_element_by_class_name("searchlistitem").text
    print(title)
    link = item.find_element_by_tag_name("a")
    print(link.get_attribute('href'))
    link.click()
    print("On page, going back")
    driver.find_element_by_partial_link_text("Return to List").click()
    print("Okay, back")


----
VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D
On page, going back
Okay, back
----


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.12.6 x86_64)


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [11]:
items = driver.find_elements_by_class_name("searchfieldtitle")
shops = []
for item in items:
    shop = {}
    shop['name'] = item.find_element_by_class_name("searchlistitem").text
    try:
        shop['url'] = item.find_element_by_tag_name("a").get_attribute('href')
    except:
        pass
    shops.append(shop)
shops

[{'name': 'VAPE IT STORE II',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D'},
 {'name': 'VAPE IT STORE I',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=NXzECYG7G0U%3D'},
 {'name': 'VAPEPAD THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=dm%2FAmpglke0%3D'},
 {'name': 'VAPE FROG'},
 {'name': 'VAPE FROG'}]

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [12]:
#try:
shops = []
while True:
    print("Scraping a new page")
    items = driver.find_elements_by_class_name("searchfieldtitle")
    for item in items:
        shop = {}
        shop['name'] = item.find_element_by_class_name("searchlistitem").text
        try:
            shop['url'] = item.find_element_by_tag_name("a").get_attribute('href')
        except:
            pass
        shops.append(shop)
    next_link = driver.find_element_by_partial_link_text('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_link)
    next_link.click()
# except:
#     print("Done")

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.12.6 x86_64)


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [17]:
# I'm cheating by clicking the first page to take me back
shops = []
while True:
    print("Scraping a new page")
    items = driver.find_elements_by_class_name("searchfieldtitle")
    for item in items:
        shop = {}
        shop['name'] = item.find_element_by_class_name("searchlistitem").text
        try:
            shop['url'] = item.find_element_by_tag_name("a").get_attribute('href')
        except:
            pass
        shops.append(shop)
    next_link = driver.find_element_by_partial_link_text('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_link)
    next_link.click()

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.12.6 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [30]:
# I'm cheating by clicking the first page to take me back
try:
    shops = []
    while True:
        print("Scraping a new page")
        items = driver.find_elements_by_class_name("searchfieldtitle")
        for item in items:
            shop = {}
            shop['name'] = item.find_element_by_class_name("searchlistitem").text
            try:
                shop['url'] = item.find_element_by_tag_name("a").get_attribute('href')
            except:
                pass
            shops.append(shop)
        next_link = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_link)
        next_link.click()
except:
    print("Done")

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Done


### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [31]:
import pandas as pd

df = pd.DataFrame(shops)
df.head()

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=NXzECYG7G0U%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=dm%2FAmpglke0%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN


In [32]:
df.to_csv("vape-shops-basic.csv", index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [33]:
pd.set_option('display.max_colwidth', -1)
df.head(1)

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D


### Use Selenium to visit that page

In [43]:
# .iloc[0] means "give me the first row", kind of like .head(1), but
# it allows me to use .url to pull out the actual url. I only did this
# so that I could use a variable - you just needed to cut and paste
url = df.iloc[0].url
url

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D'

In [44]:
driver.get(url)

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [45]:
data = {}
data['mailing'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
data['location'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
data['license'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
data['issued_by'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
data['amount_paid'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
data

{'amount_paid': '$ 41.00',
 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198',
 'license': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'location': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'mailing': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804'}

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [47]:
driver.get(url)

data = {}
data['mailing'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
data['location'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
data['license'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
data['issued_by'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
data['amount_paid'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
data

{'amount_paid': '$ 41.00',
 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198',
 'license': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'location': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'mailing': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804'}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [59]:
def get_vape_data(row):
    # Only wrap this part in a try/except - we want
    # to know if the real scraping fails, but it's
    # okay if it fails because the URL is missing/bad
    try:
        driver.get(row['url'])
    except:
        return pd.Series({})

    data = {}
    data['mailing'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
    data['location'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
    data['license'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
    data['issued_by'] = driver.find_elements_by_class_name('tablecelltext')[-2].text
    data['amount_paid'] = driver.find_elements_by_class_name('rt')[-1].text
    return pd.Series(data)

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [60]:
new_df = df.apply(get_vape_data, axis=1).join(df)
new_df

,amount_paid,issued_by,license,location,mailing,name,url
0,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,"VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801","AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pwU9fOwbeI8%3D
1,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,"VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801","AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804",VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=NXzECYG7G0U%3D
2,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,"VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113",VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=dm%2FAmpglke0%3D
3,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
4,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
5,$ 154.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",License Status: Issued\nLicense No.: 02310799\nControl No.: 02686069\nDate of Issue: 4/03/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,"VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWATER, MD 21037",DISBROW II EMERSON HARRINGTON,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=FAhM33su85I%3D
6,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045",License Status: Issued\nLicense No.: 13343011\nControl No.: 13856368\nDate of Issue: 4/30/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 13 Howard County,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045",DISCOUNT TOBACCO ESSEX LLC,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=yfAMmxvqvp8%3D
7,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155",License Status: Issued\nLicense No.: 06327188\nControl No.: 06946760\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 06 Westminster,"VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15\nWESTMINSTER, MD 21157",FAIRGROUND VILLAGE LLC,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=rXtCvFiHadU%3D
8,NaN,NaN,NaN,NaN,NaN,VAPE HAVEN,NaN
9,NaN,NaN,NaN,NaN,NaN,VAPORS LOUNGE,NaN


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [61]:
new_df.to_csv("vape-total.csv", index=False)